<a href="https://colab.research.google.com/github/shrutisingh004/Deep-Learning/blob/main/BPTT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
np.random.seed(0)

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def dsigmoid(y):
    return y * (1 - y)

In [ ]:
input_size = 1
hidden_size = 5
output_size = 1
learning_rate = 0.1
sequence_length = 4

In [ ]:
Wxh = np.random.randn(hidden_size, input_size) * 0.1
Whh = np.random.randn(hidden_size, hidden_size) * 0.1
Why = np.random.randn(output_size, hidden_size) * 0.1

bh = np.zeros((hidden_size, 1))
by = np.zeros((output_size, 1))

In [ ]:
inputs = [np.array([[0]]), np.array([[1]]), np.array([[0]]), np.array([[1]])]
targets = [np.array([[1]]), np.array([[0]]), np.array([[1]]), np.array([[0]])]

In [ ]:
for epoch in range(1000):
    hs = {}
    hs[-1] = np.zeros((hidden_size, 1))
    ys = {}
    loss = 0
    for t in range(sequence_length):
        x = inputs[t]
        hs[t] = sigmoid(np.dot(Wxh, x) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = sigmoid(np.dot(Why, hs[t]) + by)
        loss += 0.5 * np.square(ys[t] - targets[t])

    dWxh = np.zeros_like(Wxh)
    dWhh = np.zeros_like(Whh)
    dWhy = np.zeros_like(Why)
    dbh = np.zeros_like(bh)
    dby = np.zeros_like(by)
    dh_next = np.zeros_like(hs[0])

    for t in reversed(range(sequence_length)):
        dy = (ys[t] - targets[t]) * dsigmoid(ys[t])
        dWhy += np.dot(dy, hs[t].T)
        dby += dy

        dh = np.dot(Why.T, dy) + dh_next
        dh_raw = dh * dsigmoid(hs[t])
        dbh += dh_raw
        dWxh += np.dot(dh_raw, inputs[t].T)
        dWhh += np.dot(dh_raw, hs[t-1].T)
        dh_next = np.dot(Whh.T, dh_raw)

    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -1, 1, out=dparam)

    Wxh -= learning_rate * dWxh
    Whh -= learning_rate * dWhh
    Why -= learning_rate * dWhy
    bh -= learning_rate * dbh
    by -= learning_rate * dby

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {np.sum(loss):.4f}")

Epoch 0, Loss: 0.5017
Epoch 100, Loss: 0.4963
Epoch 200, Loss: 0.4850
Epoch 300, Loss: 0.4491
Epoch 400, Loss: 0.3657
Epoch 500, Loss: 0.2344
Epoch 600, Loss: 0.1187
Epoch 700, Loss: 0.0609
Epoch 800, Loss: 0.0361
Epoch 900, Loss: 0.0242


In [ ]:
print("\nTesting after training:")
hs[-1] = np.zeros((hidden_size, 1))
for t in range(sequence_length):
    hs[t] = sigmoid(np.dot(Wxh, inputs[t]) + np.dot(Whh, hs[t-1]) + bh)
    y = sigmoid(np.dot(Why, hs[t]) + by)
    print(f"Input: {inputs[t].flatten()[0]} → Predicted: {round(y.flatten()[0], 3)}")


Testing after training:
Input: 0 → Predicted: 0.908
Input: 1 → Predicted: 0.095
Input: 0 → Predicted: 0.909
Input: 1 → Predicted: 0.097
